# 🎬 Audio Transcription & Subtitle Generator

Notebook ini digunakan untuk:
1. **Mentranscribe audio** dari Google Drive menjadi subtitle (`.srt`)
2. **Dua engine** tersedia:
   - 🌍 **OpenAI Whisper Large v3** — untuk semua bahasa
   - 🇯🇵 **Kotoba-Whisper v2.2** — khusus bahasa Jepang (lebih akurat)
3. **Translate subtitle** ke Bahasa Indonesia menggunakan **Llama-3.1-8B-Instruct** (AI generatif, gaya bahasa bisa dikustomisasi)
4. **Simpan hasil** ke Google Drive

---
### 📂 Struktur Folder (otomatis dibuat)
```
MyDrive/
└── Subtitle Generator/
    ├── Audio/      ← taruh file audio di sini
    └── Output/     ← hasil subtitle disimpan di sini
```

⚠️ **Pastikan runtime GPU aktif**: `Runtime > Change runtime type > GPU (T4)`

## 📦 Cell 1: Install Dependencies

In [ ]:
# ============================================================
# 📦 Install semua dependencies yang dibutuhkan
# ============================================================

# --- OpenAI Whisper ---
!pip install -q openai-whisper

# --- Kotoba-Whisper (via Transformers) ---
!pip install -q --upgrade transformers accelerate torchaudio
!pip install -q "punctuators==0.0.5"

# --- Llama 3.1 Translation (via Transformers + 4-bit quantization) ---
!pip install -q --upgrade bitsandbytes

# --- Utilities ---
!pip install -q tqdm

print("\n✅ Semua dependencies berhasil diinstall!")

## ⚙️ Cell 2: Konfigurasi

In [ ]:
# ============================================================
# ⚙️ KONFIGURASI — Sesuaikan parameter di bawah ini
# ============================================================

#@title 🎛️ Setup Konfigurasi
#@markdown ### 🤖 Engine Transcription
engine = "openai-whisper-large-v3" #@param ["openai-whisper-large-v3", "kotoba-whisper-v2.2"]

#@markdown ---
#@markdown ### 🌐 Translate ke Bahasa Indonesia?
translate_to_id = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### ✍️ Gaya Bahasa Terjemahan
#@markdown Atur gaya penerjemahan (hanya berlaku jika translate aktif)
translation_style = "Terjemahkan ke bahasa Indonesia yang santai, natural, dan tidak kaku. Gunakan bahasa sehari-hari yang mudah dipahami." #@param {type:"string"}

# --- Fixed folder structure ---
DRIVE_BASE = "/content/drive/MyDrive"
BASE_DIR = f"{DRIVE_BASE}/Subtitle Generator"
AUDIO_DIR = f"{BASE_DIR}/Audio"
OUTPUT_DIR = f"{BASE_DIR}/Output"

print(f"📂 Base Folder   : {BASE_DIR}")
print(f"🎵 Folder Audio  : {AUDIO_DIR}")
print(f"📁 Folder Output : {OUTPUT_DIR}")
print(f"🤖 Engine        : {engine}")
print(f"🌐 Translate     : {'Ya (→ Bahasa Indonesia)' if translate_to_id else 'Tidak'}")
if translate_to_id:
    print(f"✍️ Gaya bahasa  : {translation_style}")

## 📁 Cell 3: Mount Google Drive & Cari Audio

In [ ]:
# ============================================================
# 📁 Mount Google Drive, Create Folder Structure & Discover Audio
# ============================================================

import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Auto-create folder structure if not exists
for folder in [BASE_DIR, AUDIO_DIR, OUTPUT_DIR]:
    os.makedirs(folder, exist_ok=True)
    if os.path.isdir(folder):
        print(f"✅ Folder siap: {folder}")
    else:
        print(f"❌ Gagal membuat folder: {folder}")

# Supported audio extensions
AUDIO_EXTENSIONS = {'.mp3', '.wav', '.flac', '.m4a', '.ogg', '.wma', '.aac', '.opus', '.webm'}

# Cari semua file audio di folder
all_audio_files = sorted([
    f for f in os.listdir(AUDIO_DIR)
    if os.path.splitext(f)[1].lower() in AUDIO_EXTENSIONS
])

if not all_audio_files:
    print(f"\n⚠️ Tidak ada file audio ditemukan di: {AUDIO_DIR}")
    print(f"   Silakan upload file audio ke folder tersebut di Google Drive,")
    print(f"   lalu jalankan ulang cell ini.")
    print(f"\n   Extensions yang didukung: {', '.join(sorted(AUDIO_EXTENSIONS))}")
else:
    print(f"\n🎵 Ditemukan {len(all_audio_files)} file audio:\n")
    for i, f in enumerate(all_audio_files, 1):
        size_mb = os.path.getsize(os.path.join(AUDIO_DIR, f)) / (1024 * 1024)
        print(f"   {i}. {f}  ({size_mb:.1f} MB)")
    print(f"\n📁 Output akan disimpan ke: {OUTPUT_DIR}")

## 🎯 Cell 4: Pilih Audio yang Akan Diproses

In [ ]:
# ============================================================
# 🎯 Pilih Audio yang Akan Ditranscribe
# ============================================================

import ipywidgets as widgets
from IPython.display import display

if not all_audio_files:
    raise FileNotFoundError("❌ Tidak ada file audio! Upload dulu ke folder Audio di Drive, lalu jalankan ulang Cell 3.")

# Create checkboxes for each audio file
print("🎯 Centang file audio yang ingin di-transcribe:\n")

checkboxes = {}
for f in all_audio_files:
    size_mb = os.path.getsize(os.path.join(AUDIO_DIR, f)) / (1024 * 1024)
    cb = widgets.Checkbox(
        value=True,
        description=f"{f} ({size_mb:.1f} MB)",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')
    )
    checkboxes[f] = cb

# Select All / Deselect All buttons
btn_select_all = widgets.Button(description='✅ Pilih Semua', button_style='success', layout=widgets.Layout(width='150px'))
btn_deselect_all = widgets.Button(description='❌ Hapus Semua', button_style='danger', layout=widgets.Layout(width='150px'))

def select_all(b):
    for cb in checkboxes.values():
        cb.value = True

def deselect_all(b):
    for cb in checkboxes.values():
        cb.value = False

btn_select_all.on_click(select_all)
btn_deselect_all.on_click(deselect_all)

# Display widgets
display(widgets.HBox([btn_select_all, btn_deselect_all]))
print()
for cb in checkboxes.values():
    display(cb)

print("\n⬆️ Centang/uncentang file di atas, lalu lanjutkan ke cell berikutnya.")

## ✅ Cell 5: Konfirmasi Pilihan Audio

In [ ]:
# ============================================================
# ✅ Konfirmasi file audio yang dipilih
# ============================================================

audio_files = [f for f, cb in checkboxes.items() if cb.value]

if not audio_files:
    raise ValueError("❌ Tidak ada file yang dipilih! Kembali ke cell sebelumnya dan centang minimal 1 file.")

print(f"✅ {len(audio_files)} file dipilih untuk diproses:\n")
for i, f in enumerate(audio_files, 1):
    size_mb = os.path.getsize(os.path.join(AUDIO_DIR, f)) / (1024 * 1024)
    print(f"   {i}. {f}  ({size_mb:.1f} MB)")

print(f"\n🤖 Engine: {engine}")
print(f"🌐 Translate: {'Ya (→ ID)' if translate_to_id else 'Tidak'}")
if translate_to_id:
    print(f"✍️ Gaya   : {translation_style[:60]}...")
print(f"\n▶️ Lanjutkan ke cell berikutnya untuk mulai proses.")

## 🔧 Cell 6: Setup Translation Engine (Llama-3.1-8B)

In [ ]:
# ============================================================
# 🔧 Setup Llama-3.1-8B-Instruct — Translation Engine
# ============================================================

if translate_to_id:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

    LLAMA_MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

    print(f"🤖 Loading {LLAMA_MODEL_ID} (4-bit quantization)...")
    print(f"   ✍️ Gaya bahasa: {translation_style}")

    # 4-bit quantization config agar muat di GPU T4 (16GB VRAM)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    llama_tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL_ID)
    llama_model = AutoModelForCausalLM.from_pretrained(
        LLAMA_MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
    )

    print("\n✅ Llama-3.1-8B-Instruct loaded!")

    # --- Helper: Translate function ---
    def translate_text(text):
        """
        Translate text menggunakan Llama-3.1-8B-Instruct.
        Gaya bahasa ditentukan oleh variabel translation_style.
        """
        if not text or not text.strip():
            return text

        messages = [
            {
                "role": "system",
                "content": (
                    f"{translation_style} "
                    "Kamu adalah penerjemah subtitle profesional. "
                    "Hanya berikan hasil terjemahan saja, tanpa penjelasan atau komentar tambahan. "
                    "Jangan tambahkan tanda kutip di sekitar hasil terjemahan."
                ),
            },
            {
                "role": "user",
                "content": f"Terjemahkan teks subtitle berikut ke Bahasa Indonesia:\n\n{text}",
            },
        ]

        # Tokenize menggunakan chat template
        input_ids = llama_tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(llama_model.device)

        # Generate
        with torch.no_grad():
            output_ids = llama_model.generate(
                input_ids,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.3,
                top_p=0.9,
            )

        # Decode hanya token baru (skip input)
        new_tokens = output_ids[0][input_ids.shape[-1]:]
        output = llama_tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

        # Bersihkan tanda kutip jika ada
        if output.startswith('"') and output.endswith('"'):
            output = output[1:-1]
        if output.startswith("'") and output.endswith("'"):
            output = output[1:-1]

        return output.strip()

    print("✅ Fungsi translate siap digunakan.")
else:
    print("⏭️ Translation dinonaktifkan, skip setup.")

## 🎤 Cell 7: Load Transcription Engine

In [ ]:
# ============================================================
# 🎤 Transcription Engine & SRT Generation
# ============================================================

import os
from tqdm.auto import tqdm

# ---- SRT Helper Functions ----

def format_timestamp(seconds):
    """Convert seconds to SRT timestamp format: HH:MM:SS,mmm"""
    if seconds is None:
        return "00:00:00,000"
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"


def write_srt(segments, filepath):
    """Write segments to SRT file.
    Each segment: {'start': float, 'end': float, 'text': str}
    """
    with open(filepath, 'w', encoding='utf-8') as f:
        for i, seg in enumerate(segments, 1):
            start = format_timestamp(seg['start'])
            end = format_timestamp(seg['end'])
            text = seg['text'].strip()
            f.write(f"{i}\n{start} --> {end}\n{text}\n\n")


# ---- Load Transcription Engine ----

print(f"🤖 Loading engine: {engine}...\n")

if engine == "openai-whisper-large-v3":
    import whisper
    import torch

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"   Device: {device}")
    print(f"   Loading Whisper large-v3... (ini bisa memakan waktu beberapa menit)")
    whisper_model = whisper.load_model("large-v3", device=device)
    print("   ✅ Model loaded!\n")

    def transcribe_audio(audio_path):
        """Transcribe using OpenAI Whisper large-v3"""
        result = whisper_model.transcribe(
            audio_path,
            word_timestamps=False,
            verbose=False
        )
        segments = [{
            'start': seg['start'],
            'end': seg['end'],
            'text': seg['text']
        } for seg in result['segments']]

        detected_lang = result.get('language', 'unknown')
        return segments, detected_lang

elif engine == "kotoba-whisper-v2.2":
    import torch
    from transformers import pipeline

    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model_kwargs = {"attn_implementation": "sdpa"} if torch.cuda.is_available() else {}

    print(f"   Device: {device}")
    print(f"   Loading Kotoba-Whisper v2.2... (ini bisa memakan waktu beberapa menit)")

    kotoba_pipe = pipeline(
        "automatic-speech-recognition",
        model="kotoba-tech/kotoba-whisper-v2.2",
        torch_dtype=torch_dtype,
        device=device,
        model_kwargs=model_kwargs,
        chunk_length_s=15,
        batch_size=8,
        trust_remote_code=True,
    )
    print("   ✅ Model loaded!\n")

    def transcribe_audio(audio_path):
        """Transcribe using Kotoba-Whisper v2.2"""
        result = kotoba_pipe(
            audio_path,
            return_timestamps=True,
        )

        segments = []
        if 'chunks' in result:
            for chunk in result['chunks']:
                ts = chunk.get('timestamp', (0, 0))
                start = ts[0] if ts[0] is not None else 0
                end = ts[1] if ts[1] is not None else start + 5
                segments.append({
                    'start': start,
                    'end': end,
                    'text': chunk.get('text', '')
                })
        else:
            segments.append({
                'start': 0,
                'end': 0,
                'text': result.get('text', '')
            })

        return segments, "ja"

print("="*60)
print("🚀 Engine siap! Mulai proses transcription...")
print("="*60)

## 🔄 Cell 8: Proses Audio yang Dipilih

In [ ]:
# ============================================================
# 🔄 Process Selected Audio Files → Transcribe + Translate + Save
# ============================================================

import time

results_summary = []

print(f"🔄 Memproses {len(audio_files)} file yang dipilih...\n")

for idx, audio_file in enumerate(audio_files, 1):
    audio_path = os.path.join(AUDIO_DIR, audio_file)
    base_name = os.path.splitext(audio_file)[0]

    print(f"\n{'='*60}")
    print(f"🎵 [{idx}/{len(audio_files)}] Processing: {audio_file}")
    print(f"{'='*60}")

    # --- Step 1: Transcribe ---
    print(f"\n   🎤 Transcribing...")
    start_time = time.time()

    try:
        segments, detected_lang = transcribe_audio(audio_path)
    except Exception as e:
        print(f"   ❌ Error saat transcribe: {e}")
        results_summary.append({'file': audio_file, 'status': 'ERROR', 'error': str(e)})
        continue

    elapsed = time.time() - start_time
    print(f"   ✅ Transcription selesai! ({elapsed:.1f}s)")
    print(f"   🌐 Bahasa terdeteksi: {detected_lang}")
    print(f"   📝 Total segments: {len(segments)}")

    # Preview first 3 segments
    print(f"\n   📋 Preview (3 segment pertama):")
    for seg in segments[:3]:
        print(f"      [{format_timestamp(seg['start'])} → {format_timestamp(seg['end'])}] {seg['text'][:80]}")

    # --- Step 2: Save Original SRT ---
    srt_original_path = os.path.join(OUTPUT_DIR, f"{base_name}.srt")
    write_srt(segments, srt_original_path)
    print(f"\n   💾 Saved original: {srt_original_path}")

    result_entry = {
        'file': audio_file,
        'status': 'OK',
        'language': detected_lang,
        'segments': len(segments),
        'original_srt': srt_original_path,
        'translated_srt': None,
        'time_transcribe': elapsed
    }

    # --- Step 3: Translate (if enabled) ---
    if translate_to_id:
        print(f"\n   🌐 Translating to Bahasa Indonesia (Llama-3.1-8B)...")
        print(f"   ✍️ Gaya: {translation_style[:60]}...")
        start_time = time.time()

        # Cek apakah audio sudah dalam Bahasa Indonesia
        src_lang = detected_lang.lower()
        if src_lang in ("id", "indonesian"):
            print("   ⏭️ Audio sudah dalam Bahasa Indonesia, skip translation.")
        else:
            translated_segments = []
            for i, seg in enumerate(tqdm(segments, desc="   Translating")):
                translated_text = translate_text(seg['text'])
                translated_segments.append({
                    'start': seg['start'],
                    'end': seg['end'],
                    'text': translated_text
                })

            elapsed_tr = time.time() - start_time
            print(f"   ✅ Translation selesai! ({elapsed_tr:.1f}s)")

            # Preview translated
            print(f"\n   📋 Preview terjemahan (3 segment pertama):")
            for seg in translated_segments[:3]:
                print(f"      [{format_timestamp(seg['start'])} → {format_timestamp(seg['end'])}] {seg['text'][:80]}")

            # Save translated SRT
            srt_translated_path = os.path.join(OUTPUT_DIR, f"{base_name}_id.srt")
            write_srt(translated_segments, srt_translated_path)
            print(f"\n   💾 Saved translated: {srt_translated_path}")
            result_entry['translated_srt'] = srt_translated_path

    results_summary.append(result_entry)

print(f"\n\n{'='*60}")
print(f"🏁 SEMUA FILE SELESAI DIPROSES!")
print(f"{'='*60}")

## 📊 Cell 9: Ringkasan Hasil

In [ ]:
# ============================================================
# 📊 Ringkasan Hasil
# ============================================================

print("\n" + "="*60)
print("📊 RINGKASAN HASIL")
print("="*60)
print(f"\n🤖 Engine        : {engine}")
print(f"📂 Input Folder  : {AUDIO_DIR}")
print(f"📁 Output Folder : {OUTPUT_DIR}")
print(f"🌐 Translate     : {'Ya (→ ID)' if translate_to_id else 'Tidak'}")
print(f"\n{'─'*60}")

success_count = 0
error_count = 0

for r in results_summary:
    print(f"\n🎵 {r['file']}")
    if r['status'] == 'OK':
        success_count += 1
        print(f"   ✅ Status    : Berhasil")
        print(f"   🌐 Bahasa   : {r['language']}")
        print(f"   📝 Segments : {r['segments']}")
        print(f"   ⏱️  Waktu    : {r['time_transcribe']:.1f}s")
        print(f"   📄 Original : {r['original_srt']}")
        if r['translated_srt']:
            print(f"   📄 Translated: {r['translated_srt']}")
    else:
        error_count += 1
        print(f"   ❌ Status: Error — {r.get('error', 'Unknown')}")

print(f"\n{'─'*60}")
print(f"\n📈 Total: {success_count} berhasil, {error_count} gagal dari {len(results_summary)} file")
print(f"\n📂 Semua file subtitle tersimpan di:")
print(f"   {OUTPUT_DIR}")

# List all files in output directory
if os.path.isdir(OUTPUT_DIR):
    output_files = sorted(os.listdir(OUTPUT_DIR))
    if output_files:
        print(f"\n📋 File di output folder:")
        for f in output_files:
            fpath = os.path.join(OUTPUT_DIR, f)
            size_kb = os.path.getsize(fpath) / 1024
            print(f"   • {f}  ({size_kb:.1f} KB)")

print(f"\n✨ Selesai! Silakan cek Google Drive Anda.")